# Processing Earthquake Events Data with `obspy` and `csp`

## Introduction

In this example, we will use CSP to process earthquake events and plot them in a map using a [Perspective](https://perspective.finos.org) widget.

First, we need to install a few extra libraries:
- `obspy`, for reading the earthquake stream from USGS;
- `perspective-python`, to create the visualization of the data; and 
- `cartopy`, for plotting the individual events in the USGS catalog (this is optional).

**Note:** This example has been tested for `jupyterlab==4.2.0` and `perspective-python==2.10.0`.

You can install these dependencies in your Python environment with the following command:

```
pip install obspy cartopy jupyterlab==4.2.0 perspective-python==2.10.0
```

#### Reading realtime data from USGS as QuakeML

The [USGS website](https://earthquake.usgs.gov/earthquakes) provides several feeds with recent seismic events, and we will read the "All day" feed containing all seismic events of the past 24h. Using `obspy`, we can read the feed in the [QUAKEML](https://earthquake.usgs.gov/earthquakes/feed/v1.0/quakeml.php) format as a `catalog`:

In [ ]:
from obspy import read_events

url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.quakeml"
catalog = read_events(url, format="QUAKEML")

In [ ]:
catalog

Now, we can also use [cartopy](https://scitools.org.uk/cartopy/docs/latest/) to to plot all events in a world map:

In [ ]:
catalog.plot();

To see what these event objects are, we can look closer:

In [ ]:
event = catalog[0]
event

This feed lists different kinds of events, noted by `event.event_type`. We can also inspect location, time and magnitude information:

In [ ]:
event.origins, event.magnitudes

This feed is updated every minute, meaning we get a historical dataset of all seismic events in the past 24h, but we also get a continually updated feed that adds new events as they are entered into this feed (all events will contain this `creation_time` information).

## Using CSP to Process Historical and Realtime Events

We can now use CSP to read the same data in either realtime or simulation modes, by building a `PushPullAdapter`. This adapter combines a realtime or `PushAdapter` and a historical or `PullAdapter` into a single implementation. This makes it easy to switch from historical mode to realtime mode at runtime. 

Because we want to visualize the results, we will start by creating a Perspective widget containing a table and a world map. This widget will be updated every time a new event is detected (and the corresponding CSP edge is *ticked*). We will read historical events from the past 6h, and then run the engine in realtime mode for 10 minutes while we wait for new events to be added to the catalog

First, we will create our Perspective widget to display the live updating map.

In [ ]:
from perspective import PerspectiveWidget, Plugin
import ipywidgets as widgets
from datetime import datetime

# Data schema for Perspective widget
data = {
    "longitude": float,
    "latitude": float,
    "magnitude": float,
    "time": datetime,
}

datagrid = PerspectiveWidget(
    data,
    plugin=Plugin.GRID,
    group_by=["time"],
    columns=["time", "longitude", "latitude", "magnitude"],
    aggregates={
        "time": "last",
        "longitude": "last",
        "latitude": "last",
        "magnitude": "last",
    },
)

worldmap = PerspectiveWidget(
    data,
    plugin=Plugin.MAP_SCATTER,
    columns=["longitude", "latitude", "magnitude", "magnitude", "time", "time"],
)

# Create a tab widget with some PerspectiveWidgets inside
widget = widgets.Tab()
widget.children = [datagrid, worldmap]
widget.titles = ["All events", "World map"]
widget

After launching the Widget, we see that it is empty. We will pass the Widget to our `csp` application which will update the data. 

Next, we create a `PushPullInputAdapter` to bring in the earthquake data to a `csp.graph`. In `csp`, a *push* adapter pushes real-time events to the application and a *pull* adapter pulls in historical data. A push-pull adapter will pull in historical data until its source is exhausted and then transition to real-time mode on a live feed. 

The push-pull adapter is especially useful when real-time execution depends on some *state* influenced by prior events. We can playback the history to reach our desired starting state before processing live data. In this example, we will playback the past day of earthquake events to get some data on our map before listening for new 

In [ ]:
# PushPullAdapter
import threading
import csp
from obspy import read_events
from datetime import datetime, timedelta, timezone
from csp.impl.pushpulladapter import PushPullInputAdapter
from csp.impl.wiring import py_pushpull_adapter_def
import time

# We use a csp.Struct to store the earthquake event data
class EventData(csp.Struct):
    time: datetime
    longitude: float
    latitude: float
    magnitude: float
    
# Create a runtime implementation of the adapter
class EarthquakeEventAdapter(PushPullInputAdapter):
    def __init__(self, interval, url):
        self._interval = interval
        self._thread = None
        self._running = False
        self._url = url

    def start(self, starttime, endtime):
        print("EarthquakeEventAdapter::start")
        self._running = True
        self._thread = threading.Thread(target=self._run)
        self._thread.start()
        self._starttime = starttime
        self._endtime = endtime

    def stop(self):
        print("EarthquakeEventAdapter::stop")
        if self._running:
            self._running = False
            self._thread.join()

    def _run(self):
        # This is the function that defines how data is pushed/pulled into the graph
        # First, we "pull" all the historical events in playback mode
        catalog = read_events(self._url, format="QUAKEML")
        catalog.events.sort(key=lambda event: event.origins[0].time)
        for event in catalog:
            event_data = EventData(
                time=event.origins[0].time.datetime,
                longitude=event.origins[0].longitude,
                latitude=event.origins[0].latitude,
                magnitude=event.magnitudes[0].mag,
            )
            # push_tick for a push-pull adapter takes 3 arguments: live (bool), time, value
            # for historical data live=False
            self.push_tick(False, event_data.time, event_data)

        print("-------------------------------------------------------------------")
        print(f"{datetime.utcnow()}: Historical replay complete, pulled {len(catalog)} events")
        print("-------------------------------------------------------------------")
        self.flag_replay_complete()

        last_event_time_pushed = catalog[-1].origins[0].time.datetime

        # Now we transition to live execution
        # The while-loop will run every 1-minute in real-time mode
        while self._running:
            catalog = read_events(self._url, format="QUAKEML")
            catalog.events.sort(key=lambda event: event.origins[0].time)

            # Find any new events from the last minute
            new_events = []
            for event in reversed(catalog):
                if event.origins[0].time.datetime > last_event_time_pushed:
                    new_events.append(event)
                else:
                    break

            print("-------------------------------------------------------------------")
            print(f"{datetime.utcnow()}: Refreshing earthquake live feed with {len(new_events)} events")
            print("-------------------------------------------------------------------")
            
            for event in reversed(new_events):
                # Push live data
                event_data = EventData(
                    time=event.origins[0].time.datetime,
                    longitude=event.origins[0].longitude,
                    latitude=event.origins[0].latitude,
                    magnitude=event.magnitudes[0].mag,
                )
                # for historical data live=True
                last_event_time_pushed = event_data.time
                self.push_tick(True, event_data.time, event_data)

            time.sleep(self._interval.total_seconds())
            
# Create the graph-time representation of our adapter
EarthquakeEvent = py_pushpull_adapter_def("EarthquakeEventAdapter", EarthquakeEventAdapter, csp.ts[EventData], interval=timedelta, url=str)

@csp.node
def update_widget(event: csp.ts[EventData], widget: widgets.widgets.widget_selectioncontainer.Tab):
    if csp.ticked(event):
        # widget.children = [datagrid, worldmap]
        data = {
            "time": [event.time],
            "longitude": [event.longitude],
            "latitude": [event.latitude],
            "magnitude": [event.magnitude],
        }
        widget.children[0].update(data)
        widget.children[1].update(data)

@csp.graph
def earthquake_graph():
    print("Start of graph building")
    url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.quakeml"
    interval = timedelta(seconds=60)
    earthquakes = EarthquakeEvent(interval, url=url)
    update_widget(earthquakes, widget=widget)
    csp.add_graph_output("Earthquakes", earthquakes)
    print("End of graph building")

start = datetime.utcnow() - timedelta(hours=24)
end = datetime.utcnow() + timedelta(minutes=10)
csp.run(earthquake_graph, starttime=start, endtime=end, realtime=True)
print("Done.")

## Conclusion

In this example, we created a push-pull adapter to process earthquake event data. We played back a day's worth of data before seamlessly transitioning to real-time mode and processing new events. Lastly, we displayed the data in a Perspective widget which plotted each earthquake on a world map.